# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
ET.phone_home() # ok, come on, I really had to. You can't call it ET and expect me not to try to do that...

AttributeError: 'module' object has no attribute 'phone_home'

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [6]:
document = ET.parse( './data/mondial_database.xml' )

In [7]:
mondial_root = document.getroot()

In [8]:
for i, child in zip(range(10),mondial_root): # I need to visualize this tree
    print child.tag, child.attrib['name'] 

country

KeyError: 'name'

In [9]:
for i, child in zip(range(10),mondial_root): # attrib is inside the tag, names are separate tags in the body
    print child.tag + ': ' + child.find('name').text + '\n' + 'Attribs:',  child.attrib

 country: Albania
Attribs: {'memberships': 'org-BSEC org-CEI org-CD org-SELEC org-CE org-EAPC org-EBRD org-EITI org-FAO org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICCt org-Interpol org-IDA org-IFRCS org-IFC org-IFAD org-ILO org-IMO org-IMF org-IOC org-IOM org-ISO org-OIF org-ITU org-ITUC org-IDB org-MIGA org-NATO org-OSCE org-OPCW org-OAS org-OIC org-PCA org-UN org-UNCTAD org-UNESCO org-UNIDO org-UPU org-WCO org-WFTU org-WHO org-WIPO org-WMO org-UNWTO org-WTO', 'area': '28750', 'car_code': 'AL', 'capital': 'cty-Albania-Tirane'}
country: Greece
Attribs: {'memberships': 'org-AG org-BIS org-BSEC org-CD org-SELEC org-CE org-EMU org-EAPC org-EBRD org-ECB org-EIB org-CERN org-ESA org-EU org-FATF org-FAO org-IGAD org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICJ org-ICCt org-Interpol org-IDA org-IEA org-IFRCS org-IFC org-IFAD org-IHO org-ILO org-IMO org-IMSO org-IMF org-IOC org-IOM org-OIF org-ITSO org-ITU org-ITUC org-MIGA org-NATO org-NEA org-NSG org-OECD org-OSCE org-OPCW org-OAS org-

In [22]:
# Let's look at independence days
countries = [child for child in mondial_root if child.tag == 'country'] # get country list
for country in countries: # attrib is inside the tag, names are separate tags in the body
    print country.tag + ':' , country.find('name').text + ' (' + country.attrib['car_code'] + ')\n' + 'Independence Date:',\
    country.find('indep_date').text + ', from: '\
    + country.find('indep_date').attrib.get('from', '') if  country.find('indep_date') != None else 'None'

country: Albania (AL)
Independence Date: 1912-11-28, from: Ottoman Empire
country: Greece (GR)
Independence Date: 1829-01-01, from: Ottoman Empire
country: Macedonia (MK)
Independence Date: 1991-09-17, from: Yugoslavia
country: Serbia (SRB)
Independence Date: 1992-04-11, from: Yugoslavia
country: Montenegro (MNE)
Independence Date: 2006-06-03, from: SRB
country: Kosovo (KOS)
Independence Date: 2008-02-17, from: SRB
country: Andorra (AND)
Independence Date: 1278-09-08, from: 
country: France (F)
Independence Date: None
country: Spain (E)
Independence Date: 1492-01-01, from: 
country: Austria (A)
Independence Date: 1918-11-12, from: Austria-Hungary
country: Czech Republic (CZ)
Independence Date: 1993-01-01, from: Czechoslovakia
country: Germany (D)
Independence Date: 1871-01-18, from: 
country: Hungary (H)
Independence Date: 1918-11-18, from: Austria-Hungary
country: Italy (I)
Independence Date: 1861-03-17, from: 
country: Liechtenstein (FL)
Independence Date: 1806-07-12, from: Holy Roma

In [77]:
i_mortality = [(country.find('name').text, float(country.find('infant_mortality').text))\
               for country in countries if (country.find('infant_mortality') != None)]

In [79]:
i_mortality[:10]

[('Albania', 13.19),
 ('Greece', 4.78),
 ('Macedonia', 7.9),
 ('Serbia', 6.16),
 ('Andorra', 3.69),
 ('France', 3.31),
 ('Spain', 3.33),
 ('Austria', 4.16),
 ('Czech Republic', 2.63),
 ('Germany', 3.46)]

In [81]:
i_mortality.sort(key=lambda x: x[1])

In [82]:
a[:10]

[('Monaco', 1.81),
 ('Japan', 2.13),
 ('Norway', 2.48),
 ('Bermuda', 2.48),
 ('Singapore', 2.53),
 ('Sweden', 2.6),
 ('Czech Republic', 2.63),
 ('Hong Kong', 2.73),
 ('Macao', 3.13),
 ('Iceland', 3.15)]

In [89]:
# another way, because I already am missing the nice formatting of Pandas
import pandas as pd
i_mortality_df = pd.DataFrame(i_mortality,columns=['Country','Infant Mortality Rate'])

In [88]:
i_mortality_df.head(10)

,Country,Infant Mortality Rate
0,Monaco,1.81
1,Japan,2.13
2,Norway,2.48
3,Bermuda,2.48
4,Singapore,2.53
5,Sweden,2.60
6,Czech Republic,2.63
7,Hong Kong,2.73
8,Macao,3.13
9,Iceland,3.15


In [155]:
ethnic_info = [(country.find('name').text,\
  float(country.findall('population')[-1].text),\
 country.findall('ethnicgroup'))\
 for country in countries]

In [156]:
# more transformation: convert elements. Functional programming rules!
# concept: the previous returned (country, population, ethnic group XML element objects)
# What we need to do is go through each entry, and transform the ethnic group element objects
# per country into tuples labeled by country and population (possibly redundantly) 
ethnic_info_df = pd.DataFrame(reduce(lambda x,y: x+y,\
       map(lambda tup: map(lambda elem: (tup[0],tup[1], elem.text, float(elem.attrib['percentage'])), tup[2]),\
           ethnic_info)),\
             columns=['Country', 'Population', 'Ethnic Group', 'Percentage of Population'])


In [158]:
ethnic_info_df.head(10) # yay, I got it into a data frame!

,Country,Population,Ethnic Group,Percentage of Population
0,Albania,2800138,Albanian,95.0
1,Albania,2800138,Greek,3.0
2,Greece,10816286,Greek,93.0
3,Macedonia,2059794,Macedonian,64.2
4,Macedonia,2059794,Albanian,25.2
5,Macedonia,2059794,Turkish,3.9
6,Macedonia,2059794,Gypsy,2.7
7,Macedonia,2059794,Serb,1.8
8,Serbia,7120666,Serb,82.9
9,Serbia,7120666,Montenegrin,0.9


In [204]:
edf = ethnic_info_df.set_index(['Ethnic Group','Country'])
# Now set the index, to group first by ethnic group
# and then country
edf.head(7)

Population  Percentage of Population
Ethnic Group Country                                        
Albanian     Albania       2800138                      95.0
Greek        Albania       2800138                       3.0
             Greece       10816286                      93.0
Macedonian   Macedonia     2059794                      64.2
Albanian     Macedonia     2059794                      25.2
Turkish      Macedonia     2059794                       3.9
Gypsy        Macedonia     2059794                       2.7

In [206]:
# assign the population based on the proportion
edf['Group Population']= edf['Population']*edf['Percentage of Population']/100.0

In [208]:
edf.head() # it's now correctly grouped, by ethnic group, rather than by country

Population  Percentage of Population  Group Population
Ethnic Group Country                                                          
Albanian     Albania       2800138                      95.0       2660131.100
Greek        Albania       2800138                       3.0         84004.140
             Greece       10816286                      93.0      10059145.980
Macedonian   Macedonia     2059794                      64.2       1322387.748
Albanian     Macedonia     2059794                      25.2        519068.088

In [210]:
# now use groupby to compute the sum
# first reset index to allow groupby
# then drop the now irrelevant columns
# group by ethinc group, then sum, and sort on population, and finally take the head
edf.reset_index()\
.drop(['Population','Percentage of Population'],axis=1)\
.groupby('Ethnic Group')\
.sum()\
.sort(columns='Group Population',ascending=False)\
.head(10)

,Group Population
Ethnic Group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08
